In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
import torch
from transformers import (
    LlamaConfig, LlamaForCausalLM, LlamaTokenizer
)
from utils.fallacy_utils import *
from utils.format_utils import format_tokens
import pprint
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
argo_fal = [
'Appeal to Emotion',
'Red Herring',
'Hasty Generalization',
'Ad Hominem',
'Appeal to False Authority'
]

FALLACY_DEFINITIONS = {
"Ad Hominem": "a fallacy when someone attacks the others' characters or motives instead of addressing the substance of their arguments.",
"Appeal to Opinions of False Authority": "a fallacy when someone attempts to argue or persuade by referring to the opinions or statements of another questionable authority figure who lacks sufficient credibility in the discussed matter because the authority's expertise may be inadequate/irrelevant or the authority is attributed a statement that has been tweaked.",
"Red Herring": "a fallacy when someone introduces irrelevant or confusing information in arguments to diverge attention from the main topic being discussed to irrelevant issues.",
"Appeal to Emotion": "a fallacy when someone attempts to argue or persuade by using emotive language to arouse non-rational sentiments within the intended audience.",
"Hasty Generalization": "a fallacy when someone makes generalizations based on partial/incomplete observations on a small sample of the whole populations that cannot represent or generalize to other situations legitimately."
}
argo_fal = list(FALLACY_DEFINITIONS.keys())
BASELINE_TEMPLATES =[
'''Review the definitions of these fallacies,
{definitions}
Review the conversation,
{dialog}
Based on all your previous analysis, determine which of the fallacies defined above is most dominantly present in B's argument replied to A?
Output your answer in JSON format {{"fallacy": name_of_the_fallacy, "explanation": in_a_sentence_or_two}}. If none of the fallacies is found, output {{"fallacy": "No Fallacy", "explanation": in_a_sentence_or_two}}. Only output JSON.
''',
'''Based on the following definitions of fallacies,
{definitions}
Given the conversation below,
{dialog}
Determine whether any of the fallacies defined above is present in B's argument replied to A?
Output your answer in JSON format {{"fallacy": name_of_the_fallacy, "explanation": in_a_sentence_or_two}}. If none of the fallacies is found, output {{"fallacy": "No Fallacy", "explanation": in_a_sentence_or_two}}. Only output JSON.
'''
]
#about Hasty Generalization, Appeal to Emotion, Ad Hominem, Red Herring and Appeal to False Authority
def get_USER_PROMPTS(qa):
    #ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", considering B's stance, determine whether this fallacy is present or absent in B's argument."""
    #ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", review the conversation again "{dialog}", is this fallacy present in B's argument?"""
    ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", given the conversation below: \n{dialog}\n, is the fallacy defined above present or absent in B's argument?"""
    def_str = "\n".join([f"{str(i+1)}. {fal} is {FALLACY_DEFINITIONS[fal]}" for i, fal in enumerate(argo_fal)]) 
    USER_PROMPTS = {
    #0:"""Given the conversation below,\n{dialog}\nIs B in favor of or against the topic proposed by A?""".format(dialog=qa),
    #5:"""Based on all your previous analysis, output all the fallacy/fallacies present in B's argument in JSON format {{"fallacy": [name(s)_of_the_fallacy]}}. If none of the fallacies is present, output {{"fallacy": "No Fallacy"}}. Only output JSON."""
    #5:"""Based on all your previous analysis, determine which of the fallacies defined above is dominantly present in B's argument replied to A? Output the fallacy in JSON format {{"fallacy": name_of_the_fallacy}}. If none of the fallacies is present, output {{"fallacy": "No Fallacy"}}. Only output JSON."""
    5: BASELINE_TEMPLATES[1].format(definitions=def_str, dialog=qa)
    }#1:"""Considering B's stance, is it logically sound for B to draw an argument like that?""",
    def_questions = {}
    for i, (name, def_) in enumerate(FALLACY_DEFINITIONS.items()):
        def_questions[i] = ASK_SINGLE_FALLACY.format(name=name, defi=def_, dialog=qa)
    USER_PROMPTS.update(def_questions)
    return USER_PROMPTS

# Test examples
qa0="A: Is democracy the best system available?\nB: good point. why not" # no fallacy??
qa1="A: Is veganism the best for our bodies?\nB: humans are omnivores, we need both meat and vegetables." #ad homien??
qa2="A: Has anyone been on the moon?\nB: The moon is so far away, we should focus on our society." # red hering
qa3 = "A: Is global warming really an issue?\nB: Yes, all the polar-bears are dying, and we are next."
qa4 = "A: Is there a good reason for the American war on terror?\nB: If you fight once you will never stop fighting. Never start!"
qa5 = "A: Is it justified to develop nuclear energy for commercial use?\nB: The green party in Germany has the opinion, that nuclear reactors are bad for us. It is not ok" # false authority
qa6 = '''A: Should smoking at home be illegal?\nB: Home is the place of love and peace, how can you fog it with smoke?'''#appeal to emotion
qa7 = "A: Do we need a global environment task force?\nB: It is people like you who force us normal people into the dictatorship of unnecessary organizations." #ad hominem
qa8 = "A: Is it effective to censor parts of the media?\nB: It is effective to censor parts of the media, because you never know who is watching."
qa9 = "A: Do we need a global environment task force?\nB: The global taskforce for human rights is doing a good job. Thats why we need one for environment,  too."
qa10 = "A: Do we need a global environment task force?\nB: Yes, we need. There're some important enviroment issues that need global efforts to address."
qa11 = "A: Should animals have special rights?\nB: Regarding this argumentation you have the IQ of an insect. Do you think you need special rights?" #ad hominem
qa_emotion = "A: Is nuclear power a reasonable energy source?\nB: Nuclear power was the reason of death of millions of people. It should vanish from this planet."

prompts = get_USER_PROMPTS(qa5)
print(prompts)

{5: 'Based on the following definitions of fallacies,\n1. Ad Hominem is a fallacy when someone attacks the others\' characters or motives instead of addressing the substance of their arguments.\n2. Appeal to Opinions of False Authority is a fallacy when someone attempts to argue or persuade by referring to the opinions or statements of another questionable authority figure who lacks sufficient credibility in the discussed matter because the authority\'s expertise may be inadequate/irrelevant or the authority is attributed a statement that has been tweaked.\n3. Red Herring is a fallacy when someone introduces irrelevant or confusing information in arguments to diverge attention from the main topic being discussed to irrelevant issues.\n4. Appeal to Emotion is a fallacy when someone attempts to argue or persuade by using emotive language to arouse non-rational sentiments within the intended audience.\n5. Hasty Generalization is a fallacy when someone makes generalizations based on partia

# Llama2

In [3]:
model_id = './models/llama2_hf/13bf'
tokenizer = LlamaTokenizer.from_pretrained(model_id)
kwargs = {'load_in_8bit': True, 'torch_dtype':torch.float16, 'low_cpu_mem_usage':True}#'device_map':'auto', 
model = LlamaForCausalLM.from_pretrained(model_id, **kwargs)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer.encode(E_INST)
tokenizer.encode(B_INST)

[1, 518, 25580, 29962]

In [5]:
kwargs = {
    "max_new_tokens": 512,
    "do_sample" : True,
    "top_p": 0.9,#0.9, 0.5
    "temperature": 0.75,#0.75, 0.5
    "use_cache": True,
    "top_k" : 50,#50,30
    "repetition_penalty": 1.,
    "length_penalty": 1
}
def MultiturnChat(tokenizer, model, prompt, chat_history):
    #print(chat_history)
    def format_prompt(prompt, chat_history):
        SYSTEM_PROMPT = [
            '''You are a knowledgable expert in detecting fallacies in arguments. 
            Please ensure that your responses are socially unbiased and positive in nature.
            If you don't know the answer to a question, please don't share false information.
            Answer the last question.''',
            '''You are a knowledgable expert in analysing fallacies in arguments. 
            Please ensure that your responses are socially unbiased in nature.
            Your response should not be lengthy.
            Answer the last question.'''
        ]
        dialog = []
        sys_pt = {"role": "system", "content": SYSTEM_PROMPT[1]}
        usr_pt = {"role": "user", "content": prompt}
        if chat_history:
            dialog = [sys_pt] + chat_history + [usr_pt]
        else:
            dialog = [sys_pt] + [usr_pt]
        chat_history = dialog
        #print(chat_history)        
        #formatted_prompt = format_tokens(None, [dialog], tokenizer)[0]
        if dialog[0]["role"] == "system":
            dialog = [
            {
                "role": dialog[1]["role"],
                "content": B_SYS
                + dialog[0]["content"]
                + E_SYS
                + dialog[1]["content"],
            }
            ] + dialog[2:]
        assert all([msg["role"] == "user" for msg in dialog[::2]]) and all([msg["role"] == "assistant" for msg in dialog[1::2]]), (
            "model only supports 'system','user' and 'assistant' roles, "
            "starting with user and alternating (u/a/u/a/u...)")
        """
        Please verify that your tokenizer support adding "[INST]", "[/INST]" to your inputs.
        Here, we are adding it manually.
        """
        formatted_dialog = [
            f"{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} "
            for prompt, answer in zip(dialog[::2], dialog[1::2])
        ]
        
        assert (
            dialog[-1]["role"] == "user"
        ), f"Last message must be from user, got {dialog[-1]['role']}"
        formatted_dialog += [f"{B_INST} {(dialog[-1]['content']).strip()} {E_INST}"]
            
        return "".join(formatted_dialog), chat_history
    
    formatted_prompt, chat_history = format_prompt(prompt, chat_history)
    model_input = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
    print(model_input.input_ids)
    model.eval()
    with torch.no_grad():
        response = tokenizer.decode(model.generate(**model_input, **kwargs)[0], skip_special_tokens=True)
        res = response.split(E_INST)
        print("\n".join(res[:-1]))
        print()
        print(pprint.pformat(res[-1]))
        chat_history = chat_history[1:] + [{"role":"assistant", "content":res[-1]}]
        return chat_history

In [6]:
chat_history=[]

In [7]:
chat_history = MultiturnChat(tokenizer, model, prompts[0], chat_history=chat_history)

tensor([[    1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13,  3492,
           526,   263,  1073,   839, 29531, 17924,   297,  3483,   952,   292,
           285,  9864,  2478,   297,  6273, 29889, 29871,    13,  9651,  3529,
          9801,   393,   596, 20890,   526,  5374,   635,   443,  5365,  1463,
           297,  5469, 29889,    13,  9651,  3575,  2933,   881,   451,   367,
          3309, 29891, 29889,    13,  9651,   673,   278,  1833,  1139, 29889,
            13, 29966,   829, 14816, 29903,  6778,    13,    13,  7504,  3278,
           304,   278,  5023,   310,  2087,   379,  5817,   331, 29901,   376,
         29874,   285,  9864,  1270,   746,  4856, 16661,   278,  4045, 29915,
          4890,   470,  3184,  3145,  2012,   310,  3211,   292,   278,  5960,
           749,   310,  1009,  6273, 19602,  2183,   278, 14983,  2400, 29901,
         29871,    13, 29909, 29901,  1317,   372,   925,  2164,   304,  2693,
         20346,  5864,   363, 12128,   671, 29973,  

In [82]:
chat_history = MultiturnChat(tokenizer, model, prompts[5], chat_history=chat_history)

[INST] <<SYS>>
You are a knowledgable expert in analysing fallacies in arguments. 
            Please ensure that your responses are socially unbiased in nature.
            Your response should not be lengthy.
            Answer the last question.
<</SYS>>

According to the definition of Ad Hominem: "a fallacy when someone attacks the others' characters or motives instead of addressing the substance of their arguments.", given the conversation below: 
A: Is it justified to develop nuclear energy for commercial use?
B: The green party in Germany has the opinion, that nuclear reactors are bad for us. It is not ok
, is the fallacy defined above present or absent in B's argument? 
 Based on the conversation provided, the fallacy defined above (Ad Hominem) is absent in B's argument. B is not attacking the character or motives of the Green Party in Germany, but rather presenting their opinion as a reason for why nuclear energy may not be justified for commercial use. B's argument is address

In [12]:
chat_history = MultiturnChat(tokenizer, model, prompt_emo, chat_history=chat_history)

[INST] <<SYS>>
You are a knowledgable expert in analysing fallacies in arguments. 
            Please ensure that your responses are socially unbiased in nature.
            If you don't know the answer to a question, please don't share false information.
            Your response should not be lengthy.
            Answer the last question.
<</SYS>>


According to the definition of Emotional Language: "A fallacy when someone tries to to persuade his stance using emotive language to arouse non-rational emotions within the intended audience.,
given a conversation, determine whether this fallacy is present or absent in B's argument and output in JSON format.
Example 1
Conversation:
A: Is nuclear power a reasonable energy source?
B: Nuclear power was the reason of death of millions of people. It should vanish from this planet.
Answer:
{"decision": "Present", "explanation": "B's argument does commit the fallacy of Emotional Language. B attempts to arouse potential negative emotions (e.g., f

In [10]:
prompt_emo = '''
According to the definition of Emotional Language: "A fallacy when someone tries to to persuade his stance using emotive language to arouse non-rational emotions within the intended audience.,
given a conversation, determine whether this fallacy is present or absent in B's argument and output in JSON format.
Example 1
Conversation:
A: Is nuclear power a reasonable energy source?\nB: Nuclear power was the reason of death of millions of people. It should vanish from this planet.
Answer:
{{"decision": "Present", "explanation": "B's argument does commit the fallacy of Emotional Language. B attempts to arouse potential negative emotions (e.g., fear, anger, and aversion) towards nuclear power within people by stating that nuclear power has caused many deaths. B’s argument is an emotive claim that is not supported by evidence, hence it is not a reasoned argument against the use of nuclear power."}}
Example 2
A: Do we need a global environment task force?\nB: The global taskforce for human rights is doing a good job. Thats why we need one for environment, too.
Answer: 
{{"decision": "Absent", "explanation": "B's argument does not commit the fallacy of Emotional Language. B's argument does not employ emotionally charged language to persuade his stance. Therefore, the Emotional Language fallacy is absent."}}
---------------
Conversation:
{dialog}
'''.format(dialog=qa5)

In [ ]:
print(chat_history)

In [2]:
ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", given the conversation below: \n"[]"\n, is the fallacy defined above present or absent in B's argument?"""

In [4]:
tp = ASK_SINGLE_FALLACY.format(name="a", defi="b")
if "\"[]\"" in tp:
    tp = tp.replace("\"[]\"", "\"{dialog}\"")
usr_ct = tp.format(dialog='sdfadff') if '{dialog}' in tp else tp
usr_ct

'According to the definition of a: "b", given the conversation below: \n"sdfadff"\n, is the fallacy defined above present or absent in B\'s argument?'

# Llama3
## Transformers AutoModelForCausalLM

In [32]:
import transformers
import torch
from typing import List, Literal, TypedDict
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
from transformers import (BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer)
import pprint

Role = Literal["user", "assistant", "system"]

class Message(TypedDict):
    role: Role
    content: str

dialogs = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
    ]
FALLACY_DEFINITIONS = {
"Ad Hominem": "a fallacy when someone attacks the others' characters or motives instead of addressing the substance of their arguments.",
"Appeal to Opinions of False Authority": "a fallacy when someone attempts to argue or persuade by referring to the opinions or statements of another questionable authority figure who lacks sufficient credibility in the discussed matter because the authority's expertise may be inadequate/irrelevant or the authority is attributed a statement that has been tweaked.",
"Red Herring": "a fallacy when someone introduces irrelevant or confusing information in arguments to diverge attention from the main topic being discussed to irrelevant issues.",
"Appeal to Emotion": "a fallacy when someone attempts to argue or persuade by using emotive language to arouse non-rational sentiments within the intended audience.",
"Hasty Generalization": "a fallacy when someone makes generalizations based on partial/incomplete observations on a small sample of the whole populations that cannot represent or generalize to other situations legitimately."
}
argo_fal = list(FALLACY_DEFINITIONS.keys())
BASELINE_TEMPLATES =[
'''Review the definitions of these fallacies,
{definitions}
Review the conversation,
{dialog}
Based on all your previous analysis, determine which of the fallacies defined above is most dominantly present in B's argument replied to A?
Output your answer in JSON format {{"fallacy": name_of_the_fallacy, "explanation": in_a_sentence_or_two}}. If none of the fallacies is found, output {{"fallacy": "No Fallacy", "explanation": in_a_sentence_or_two}}. Only output JSON.
''',
'''Based on the following definitions of fallacies,
{definitions}
Given the conversation below,
{dialog}
Determine whether any of the fallacies defined above is present in B's argument replied to A?
Output your answer in JSON format {{"fallacy": name_of_the_fallacy, "explanation": in_a_sentence_or_two}}. If none of the fallacies is found, output {{"fallacy": "No Fallacy", "explanation": in_a_sentence_or_two}}. Only output JSON.
'''
]
#about Hasty Generalization, Appeal to Emotion, Ad Hominem, Red Herring and Appeal to False Authority
def get_USER_PROMPTS(qa):
    #ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", considering B's stance, determine whether this fallacy is present or absent in B's argument."""
    #ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", review the conversation again "{dialog}", is this fallacy present in B's argument?"""
    ASK_SINGLE_FALLACY = """According to the definition of {name}: "{defi}", given the conversation below: \n{dialog}\n, is the fallacy defined above present or absent in B's argument?"""
    def_str = "\n".join([f"{str(i+1)}. {fal} is {FALLACY_DEFINITIONS[fal]}" for i, fal in enumerate(argo_fal)]) 
    USER_PROMPTS = {
    #0:"""Given the conversation below,\n{dialog}\nIs B in favor of or against the topic proposed by A?""".format(dialog=qa),
    #5:"""Based on all your previous analysis, output all the fallacy/fallacies present in B's argument in JSON format {{"fallacy": [name(s)_of_the_fallacy]}}. If none of the fallacies is present, output {{"fallacy": "No Fallacy"}}. Only output JSON."""
    #5:"""Based on all your previous analysis, determine which of the fallacies defined above is dominantly present in B's argument replied to A? Output the fallacy in JSON format {{"fallacy": name_of_the_fallacy}}. If none of the fallacies is present, output {{"fallacy": "No Fallacy"}}. Only output JSON."""
    5: BASELINE_TEMPLATES[1].format(definitions=def_str, dialog=qa)
    }#1:"""Considering B's stance, is it logically sound for B to draw an argument like that?""",
    def_questions = {}
    for i, (name, def_) in enumerate(FALLACY_DEFINITIONS.items()):
        def_questions[i] = ASK_SINGLE_FALLACY.format(name=name, defi=def_, dialog=qa)
    USER_PROMPTS.update(def_questions)
    return USER_PROMPTS
# Test examples
qa0="A: Is democracy the best system available?\nB: good point. why not" # no fallacy??
qa1="A: Is veganism the best for our bodies?\nB: humans are omnivores, we need both meat and vegetables." #ad homien??
qa2="A: Has anyone been on the moon?\nB: The moon is so far away, we should focus on our society." # red hering
qa3 = "A: Is global warming really an issue?\nB: Yes, all the polar-bears are dying, and we are next."
qa4 = "A: Is there a good reason for the American war on terror?\nB: If you fight once you will never stop fighting. Never start!"
qa5 = "A: Is it justified to develop nuclear energy for commercial use?\nB: The green party in Germany has the opinion, that nuclear reactors are bad for us. It is not ok" # false authority
qa6 = '''A: Should smoking at home be illegal?\nB: Home is the place of love and peace, how can you fog it with smoke?'''#appeal to emotion
qa7 = "A: Do we need a global environment task force?\nB: It is people like you who force us normal people into the dictatorship of unnecessary organizations." #ad hominem
qa8 = "A: Is it effective to censor parts of the media?\nB: It is effective to censor parts of the media, because you never know who is watching."
qa9 = "A: Do we need a global environment task force?\nB: The global taskforce for human rights is doing a good job. Thats why we need one for environment,  too."
qa10 = "A: Do we need a global environment task force?\nB: Yes, we need. There're some important enviroment issues that need global efforts to address."
qa11 = "A: Should animals have special rights?\nB: Regarding this argumentation you have the IQ of an insect. Do you think you need special rights?" #ad hominem
qa_emotion = "A: Is nuclear power a reasonable energy source?\nB: Nuclear power was the reason of death of millions of people. It should vanish from this planet."

prompts = get_USER_PROMPTS(qa5)
print(prompts)

{5: 'Based on the following definitions of fallacies,\n1. Ad Hominem is a fallacy when someone attacks the others\' characters or motives instead of addressing the substance of their arguments.\n2. Appeal to Opinions of False Authority is a fallacy when someone attempts to argue or persuade by referring to the opinions or statements of another questionable authority figure who lacks sufficient credibility in the discussed matter because the authority\'s expertise may be inadequate/irrelevant or the authority is attributed a statement that has been tweaked.\n3. Red Herring is a fallacy when someone introduces irrelevant or confusing information in arguments to diverge attention from the main topic being discussed to irrelevant issues.\n4. Appeal to Emotion is a fallacy when someone attempts to argue or persuade by using emotive language to arouse non-rational sentiments within the intended audience.\n5. Hasty Generalization is a fallacy when someone makes generalizations based on partia

In [3]:
#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = './models/llama3_hf/8bf'

tokenizer = AutoTokenizer.from_pretrained(model_id)
print(tokenizer.__class__)
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = "left"
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

def format_tokens(tokenizer, dialog):
    def encode_header(message: Message) -> List[int]:
        tokens = []
        tokens.extend(tokenizer.encode("<|start_header_id|>"))
        tokens.extend(tokenizer.encode(message["role"])) #bos=False, eos=False
        tokens.extend(tokenizer.encode("<|end_header_id|>"))
        tokens.extend(tokenizer.encode("\n\n")) #bos=False, eos=False
        return tokens

    def encode_message(message: Message) -> List[int]:
        tokens = encode_header(message)
        tokens.extend(
            tokenizer.encode(message["content"].strip()) #bos=False, eos=False
        )
        tokens.extend(tokenizer.encode("<|eot_id|>"))
        return tokens
    
    prompt_tokens = []
    prompt_tokens.extend(tokenizer.encode("<|begin_of_text|>"))
    for message in dialog:
        prompt_tokens.extend(encode_message(message))
    # Add the start of an assistant message for the model to complete.
    prompt_tokens.extend(encode_header({"role": "assistant", "content": ""}))
    #print(prompt_tokens)
    return prompt_tokens

#format_tokens(tokenizer, dialog=dialogs[0])

quantization_config = BitsAndBytesConfig(load_in_8bit=False)
kwargs = {'torch_dtype':torch.bfloat16,'low_cpu_mem_usage':True, 'quantization_config': quantization_config, 'device_map':'cuda'} # , 'quantization_config': quantization_config #torch.bfloat16 is much slower then float16
# We loaded the model in bfloat16. This is the type used by the original checkpoint published by Meta, 
# so it’s the recommended way to run to ensure the best precision or to conduct evaluations. 
# For real world use, it’s also safe to use float16, which may be faster depending on your hardware.
model = AutoModelForCausalLM.from_pretrained(model_id, **kwargs)
model.eval()
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<class 'transformers.tokenization_utils_fast.PreTrainedTokenizerFast'>


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [40]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
gen_kwargs = {
    "max_new_tokens": 512,
    "eos_token_id": terminators,
    "do_sample" : True,
    "top_p": 0.9,#0.9, 0.5
    "temperature": 0.6,#0.75, 0.5
    #"use_cache": True,
    #"top_k" : 50,#50,30
    # "repetition_penalty": 1.,
    # "length_penalty": 1
}
def MultiturnChat(tokenizer, model, prompt, chat_history):
    #print(chat_history)
    def format_prompt(prompt, chat_history):
        SYSTEM_PROMPT = [
            '''You are a knowledgable expert in detecting fallacies in arguments. 
            Please ensure that your responses are socially unbiased and positive in nature.
            If you don't know the answer to a question, please don't share false information.
            Answer the last question.''',
            '''You are a knowledgable expert in analysing fallacies in arguments. 
            Please ensure that your responses are socially unbiased in nature.
            Your response should not be lengthy.
            Answer the last question.'''
        ]
        dialog = []
        sys_pt = {"role": "system", "content": SYSTEM_PROMPT[1]}
        usr_pt = {"role": "user", "content": prompt}
        if chat_history:
            dialog = [sys_pt] + chat_history + [usr_pt]
        else:
            dialog = [sys_pt] + [usr_pt]
        chat_history = dialog
        #print(dialog)
        #print("="*50)
        def format_dialog(dialog):
            def format_header(message: Message) -> List:
                tokens = []
                tokens.extend(["<|start_header_id|>"])
                tokens.extend([message["role"]]) #bos=False, eos=False
                tokens.extend(["<|end_header_id|>"])
                tokens.extend(["\n\n"]) #bos=False, eos=False
                return tokens

            def format_message(message: Message) -> List:
                tokens = format_header(message)
                tokens.extend(
                    [message["content"].strip()] #bos=False, eos=False
                )
                tokens.extend(["<|eot_id|>"])
                return tokens
            
            prompt_tokens = []
            prompt_tokens.extend(["<|begin_of_text|>"])
            for message in dialog:
                prompt_tokens.extend(format_message(message))
            # Add the start of an assistant message for the model to complete.
            prompt_tokens.extend(format_header({"role": "assistant", "content": ""}))
            #print(prompt_tokens)
            return prompt_tokens
        # #print(chat_history)        
        # formatted_prompt = format_tokens(None, [dialog], tokenizer)[0]
        # if dialog[0]["role"] == "system":
        #     dialog = [
        #     {
        #         "role": dialog[1]["role"],
        #         "content": B_SYS
        #         + dialog[0]["content"]
        #         + E_SYS
        #         + dialog[1]["content"],
        #     }
        #     ] + dialog[2:]
        # assert all([msg["role"] == "user" for msg in dialog[::2]]) and all([msg["role"] == "assistant" for msg in dialog[1::2]]), (
        #     "model only supports 'system','user' and 'assistant' roles, "
        #     "starting with user and alternating (u/a/u/a/u...)")
        # """
        # Please verify that your tokenizer support adding "[INST]", "[/INST]" to your inputs.
        # Here, we are adding it manually.
        # """
        # formatted_dialog = [
        #     f"{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} "
        #     for prompt, answer in zip(dialog[::2], dialog[1::2])
        # ]
        
        # assert (
        #     dialog[-1]["role"] == "user"
        # ), f"Last message must be from user, got {dialog[-1]['role']}"
        
        formatted_dialog = format_dialog(dialog)
        
        return "".join(formatted_dialog), chat_history
    
    formatted_prompt, chat_history = format_prompt(prompt, chat_history)
    #print(formatted_prompt)
    #print("="*50)
    model_input = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
    print(model_input.input_ids)
    model.eval()
    with torch.no_grad():
        output = model.generate(**model_input, **gen_kwargs)[0]
        response = tokenizer.decode(output, skip_special_tokens=False)  #[model_input.input_ids.shape[-1]:]
        #print(response)
        res = response.split("<|start_header_id|> assistant <|end_header_id|>")
        print("\n".join(res[:-1]))
        print()
        print(pprint.pformat(res[-1]))
        chat_history = chat_history[1:] + [{"role":"assistant", "content":res[-1]}]
        return chat_history

In [41]:
chat_history = []

In [42]:
chat_history = MultiturnChat(tokenizer, model, prompts[0], chat_history=chat_history)

tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,   1440,
            839,  36134,   6335,    304,  22209,    287,   4498,  27121,    304,
           6105,     13,    720,    310,   5321,   6106,    430,    701,  14847,
            527,  40418,  74315,    304,   7138,    627,    310,   4718,   2077,
           1288,    539,    387,  35306,    627,    310,  22559,    279,   1566,
           3488,     13, 128009, 128006,    882, 128007,    271,  11439,    311,
            279,   7419,    315,   2467,    473,   8129,    336,     25,    330,
             64,   4498,   2826,    994,   4423,   8951,    279,   3885,      6,
           5885,    477,  52140,   4619,    315,  28118,    279,  20278,    315,
            872,   6105,  10684,   2728,    279,  10652,   3770,     25,    720,
             32,     25,   2209,    433,  35516,    311,   2274,  11499,   4907,
            369,   8518,   1005,   5380,     33,     25,    578,   6307,   4717,
            304,  10057,    

## Transformers pipeline

In [2]:
pipeline = transformers.pipeline(
  "text-generation",
  model='./models/llama3_hf/8bf',
  model_kwargs={"torch_dtype": torch.bfloat16},
  device="cuda",
)
# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#     {"role": "user", "content": "Who are you?"},
# ]
messages = [
    {
        "role": "system",
        "content": "You are a knowledgable expert in analysing fallacies in arguments. \n            Please ensure that your responses are socially unbiased in nature.\n            Your response should not be lengthy.\n            Answer the last question."
    },
    {
        "role": "user", 
        "content": '''According to the definition of Ad Hominem: "a fallacy when someone attacks the others\' characters or motives instead of addressing the substance of their arguments.", given the conversation below: \nA: Is it justified to develop nuclear energy for commercial use?\nB: The green party in Germany has the opinion, that nuclear reactors are bad for us. It is not ok\n, is the fallacy defined above present or absent in B\'s argument?'''
    }
    
]

prompt = pipeline.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][:len(prompt)])
print(outputs[0]["generated_text"][len(prompt):])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a knowledgable expert in analysing fallacies in arguments. 
            Please ensure that your responses are socially unbiased in nature.
            Your response should not be lengthy.
            Answer the last question.<|eot_id|><|start_header_id|>user<|end_header_id|>

According to the definition of Ad Hominem: "a fallacy when someone attacks the others' characters or motives instead of addressing the substance of their arguments.", given the conversation below: 
A: Is it justified to develop nuclear energy for commercial use?
B: The green party in Germany has the opinion, that nuclear reactors are bad for us. It is not ok
, is the fallacy defined above present or absent in B's argument?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


The fallacy of Ad Hominem is present in B's argument. B is attacking the character of the Green Party in Germany (and by extension, its members) instead of addressing the